(content:kinematics)=
# Kinematics

## Angular Scattering

Fundamentally, in the sub-relativistic regime, the following conservation laws typically form the basis of any kinematic reconstruction
:::{math}
:label: non-relativistic-kinematics

\sum_j \vb{p^{j}_i} &= \sum_j \vb{p^{j}_f}

\sum_j E^{j}_i &= \sum_j E^{j}_f\,.
:::

In the context of a scattering reaction between an incident heavy (larger A, Z) ion and a stationary light (smaller A, Z) target (gas), the reaction will be forward projected and the range of the scattered light ion will be significantly greater than that of the recoiling heavy ion. As such, the tracks left by light ions moving through the TPC are much more easily identified, and may be fit with greater precision than fits performed using the heavy recoil. 

Given the energy and angular straggling of the beam following interactions with the window, it is also evident that indirect measurements of the beam energy will be less accurate than those of the light ion after it interacts with the silicon detector array. As such, the most accurate measurements of the reaction kinematics are given by the track scattering angles and known energy of the light product. In such a configuration, the beam and recoil momentum vectors are initially unknown. We can define a beam-centric coordinate system with the {math}`\hat{1}` vector given by the beam direction, the {math}`\hat{3}` vector given by the normalised cross product of the beam and scatter tracks, and the {math}`\hat{2}` vector taken to be perpendicular to {math}`\hat{1}` and {math}`\hat{3}`.

From {eq}`non-relativistic-kinematics`, conservation of momentum can be applied separately to each axis such that we have
:::{math}
p_1 &= p_2^\pqty{1} + p_3^\pqty{1}

0 &= p_2^\pqty{2} + p_3^\pqty{2}\,.
:::
An application of energy conservation yields
:::{math}
\frac{
    \pqty{p_1}
^2}{2m_1} =
\frac{
    \pqty{p_2}
^2}{2m_2} +
\frac{
    \pqty{p_3}
^2}{2m_3}\,.
:::


It can be shown that the recoil momentum is uniquely determined by
:::{math}
:label: angular-kinematics-recoil
p_3^\pqty{1} &= \frac{\pqty{\pqty{p_2^\pqty{2}}^2 + \pqty{p_2^\pqty{1}}^2}\frac{m_1}{m_2} + \pqty{p_2^\pqty{2}}^2 - \pqty{p_2^\pqty{1}}^2}{2 p_2^\pqty{1} }\\
&= \frac{\pqty{p_2^\pqty{2}}^2\pqty{m_1+m_2} + \pqty{p_2^\pqty{1}}^2\pqty{m_1-m_2}}{2 p_2^\pqty{1} m_2 }\\
:::


::::{tip}
:class: dropdown no-latex

:::{math}
\frac{\pqty{p_1}^2}{2m_1} &= \frac{\pqty{p_2}^2}{2m_2} + \frac{\pqty{p_3}^2}{2m_1}\\
p_1^\pqty{1} &= p_2^\pqty{1} + p_3^\pqty{1} \\
0 &= p_2^\pqty{2} + p_3^\pqty{2} \\
\frac{\left(p_2^\pqty{1} + p_3^\pqty{1}\right)^2}{2m_1} &= \frac{\pqty{p_2^\pqty{2}}^2 + \pqty{p_2^\pqty{1}}^2}{2m_2} + \frac{\pqty{p_3^\pqty{2}}^2 + \pqty{p_3^\pqty{1}}^2}{2m_1}\\
\frac{\left(p_2^\pqty{1} + p_3^\pqty{1}\right)^2}{2m_1} &= \frac{\pqty{p_2^\pqty{2}}^2 + \pqty{p_2^\pqty{1}}^2}{2m_2} + \frac{\pqty{p_2^\pqty{2}}^2 + \pqty{p_3^\pqty{1}}^2}{2m_1}\\
\frac{\pqty{p_2^\pqty{1}}^2 + 2p_2^\pqty{1}p_3^\pqty{1}}{2m_1} &= \frac{\pqty{p_2^\pqty{2}}^2 + \pqty{p_2^\pqty{1}}^2}{2m_2} + \frac{\pqty{p_2^\pqty{2}}^2}{2m_1}\\
{\pqty{p_2^\pqty{1}}^2 + 2p_2^\pqty{1}p_3^\pqty{1}} &= \left(\pqty{p_2^\pqty{2}}^2 + \pqty{p_2^\pqty{1}}^2\right)\frac{m_1}{m_2} + \pqty{p_2^\pqty{2}}^2\\
p_3^\pqty{1}&= \frac{\pqty{p_2^\pqty{2}}^2\pqty{m_1+m_2} + \pqty{p_2^\pqty{1}}^2\pqty{m_1-m_2}}{2 p_2^\pqty{1} m_2 }\\
:::

::::

This approach necessarily requires the knowledge of the interaction vertex (to identify the light particle energy) and the relative angle of the beam to the scattered ion track. If both of these cannot be determined, then there is insufficient knowledge of the system to find a solution. In the zero-degree regime, in which the light product track is close to zero degrees, it is possible to leverage the assumption of a linear reaction in order to once-again determine the beam energy (see {ref}`content:zero-degree-scattering`).

(content:zero-degree-scattering)=
## Zero-degree Scattering

In the zero-degree approximation, {eq}`angular-kinematics-recoil` simplifies to
:::{math}
p_3 = \frac{p_2\pqty{m_1 - m_2}}{2 m_2}\,.
:::
This in turn admits a relationship between the beam and scatter energies
:::{math}
\frac{E_2}{E_1} = \frac{4 m_1 m_2}{\pqty{m_1+m_2}^2}\,.
:::


::::{tip}
:class: dropdown no-latex

:::{math}
E_1 &= \frac{\pqty{p_3}^2}{2m_1} + \frac{\pqty{p_2}^2}{2m_2}

&= \frac{\pqty{p_2}^2 \pqty{\frac{m_1}{m_2}-1}^2}{4\cdot 2m_1} + \frac{\pqty{p_2}^2}{2m_2}

&= \frac{\pqty{p_2}^2}{2}\bqty{\frac{\pqty{\frac{m_1}{m_2}-1}^2}{4\cdot m_1}+\frac{1}{m_2}}

&= \frac{\pqty{p_2}^2}{2}\frac{\bqty{m_2\pqty{\frac{m_1}{m_2}-1}^2+4m_1}}{4m_1 m_2}

&= \frac{\pqty{p_2}^2}{2}\frac{\bqty{\pqty{m_2-m_1}^2+4m_1m_2}}{4m_1 m_2^2}

&= \frac{\pqty{p_2}^2}{2m_2}\frac{\pqty{m_2+m_1}^2}{4m_1 m_2}

&= E_2\frac{\pqty{m_2+m_1}^2}{4m_1 m_2}
:::

::::